# PENDAHULUAN

Pendahuluan Tugas Midterm Machine Learning – Fraud Detection

Notebook ini mengerjakan tugas UTS Machine Learning untuk membangun pipeline end-to-end deteksi fraud. Tujuannya memprediksi probabilitas suatu transaksi online bersifat fraud (isFraud) menggunakan dataset train_transaction.csv dan test_transaction.csv.

Dataset asli terdiri dari banyak fitur numerik dan beberapa fitur kategorik. Untuk menjaga agar eksekusi tetap stabil di lingkungan Google Colab, saya menggunakan sebagian baris data dan fokus pada fitur numerik. Langkah yang saya lakukan meliputi pemuatan data, pembersihan dan preprocessing, pemilihan dan pelatihan model machine learning, tuning sederhana hyperparameter, evaluasi dengan beberapa metrik, serta pembuatan file submission TransactionID dan probabilitas isFraud.

# Mount Google Drive dan load dataset

Cell ini menghubungkan Colab dengan Google Drive, lalu membaca file CSV.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

# Ganti dengan folder kamu sendiri
base_path = "/content/drive/MyDrive/DATASETML"

train_path = base_path + "/train_transaction.csv"
test_path  = base_path + "/test_transaction.csv"

# Baca sebagian baris saja supaya tidak habis RAM
# Kalau sudah lancar, boleh pelan-pelan naikkan angkanya
N_ROWS_TRAIN = 150000
N_ROWS_TEST  = 150000

train_df = pd.read_csv(train_path, nrows=N_ROWS_TRAIN)
test_df  = pd.read_csv(test_path,  nrows=N_ROWS_TEST)

print("Shape train:", train_df.shape)
print("Shape test :", test_df.shape)

print("\nLima baris pertama train:")
display(train_df.head())


Mounted at /content/drive
Shape train: (150000, 394)
Shape test : (150000, 393)

Lima baris pertama train:


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Deskripsi dataset

File train_transaction.csv berisi data transaksi dengan label isFraud (0 atau 1). Setiap baris merepresentasikan satu transaksi dengan banyak fitur seperti jumlah transaksi, informasi kartu, kode produk, dan waktu transaksi.

File test_transaction.csv berisi data transaksi tanpa label isFraud. File ini digunakan sebagai input model untuk menghasilkan prediksi probabilitas fraud yang akan ditulis ke file submission.

Pada deskripsi soal disebutkan juga identity table. Pada praktik tugas ini saya menggunakan file train_transaction.csv dan test_transaction.csv yang tersedia di Google Drive kelas. Identity table tidak disertakan sehingga pipeline difokuskan pada transaction table. Hal ini bisa menjadi pengembangan lanjutan di luar scope notebook ini.

Alasan sampling sebagian data

Dataset asli berukuran besar sehingga pemrosesan semua baris sekaligus dapat menyebabkan kehabisan RAM di Google Colab. Untuk menjaga runtime tetap stabil, saya menggunakan parameter nrows saat membaca CSV sehingga model dilatih pada subset data yang tetap merepresentasikan pola data, tetapi lebih ringan di memori.

# Cek target isFraud dan imbalance

Cell ini melihat seberapa banyak transaksi fraud dibanding normal.

In [2]:
print("Distribusi isFraud (jumlah):")
print(train_df["isFraud"].value_counts())

print("\nDistribusi isFraud (proporsi):")
print(train_df["isFraud"].value_counts(normalize=True))


Distribusi isFraud (jumlah):
isFraud
0    146030
1      3970
Name: count, dtype: int64

Distribusi isFraud (proporsi):
isFraud
0    0.973533
1    0.026467
Name: proportion, dtype: float64


# Feature engineering sederhana dan pisahkan fitur–label

Kita buat fitur tambahan log untuk TransactionAmt.
Lalu pisahkan X, y dan simpan TransactionID untuk submission.

In [3]:
# Feature engineering: log transform TransactionAmt jika ada
if "TransactionAmt" in train_df.columns:
    train_df["TransactionAmt_log"] = np.log1p(train_df["TransactionAmt"])
    test_df["TransactionAmt_log"]  = np.log1p(test_df["TransactionAmt"])
    print("Kolom TransactionAmt_log ditambahkan.")
else:
    print("Kolom TransactionAmt tidak ada, skip log transform.")

# Simpan TransactionID untuk file submission
test_ids = test_df["TransactionID"]

# y adalah label
y = train_df["isFraud"]

# X adalah semua fitur selain isFraud
X = train_df.drop(columns=["isFraud"])

# Buang TransactionID dari fitur
if "TransactionID" in X.columns:
    X = X.drop(columns=["TransactionID"])
if "TransactionID" in test_df.columns:
    X_test = test_df.drop(columns=["TransactionID"])
else:
    X_test = test_df.copy()

print("Shape X train:", X.shape)
print("Shape X test :", X_test.shape)


Kolom TransactionAmt_log ditambahkan.
Shape X train: (150000, 393)
Shape X test : (150000, 393)


# Bagi train dan validasi

Kita bagi data menjadi train dan valid dengan stratify, supaya proporsi fraud sama.

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Jumlah data train :", X_train.shape[0])
print("Jumlah data valid :", X_valid.shape[0])


Jumlah data train : 120000
Jumlah data valid : 30000


# Preprocessing numerik

Kolom numerik: isi missing dengan median, lalu scaling.

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Hanya ambil fitur numerik supaya ringan
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()

print("Jumlah fitur numerik yang dipakai:", len(numeric_features))

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
    ],
    remainder="drop"   # kolom non-numerik dibuang
)


Jumlah fitur numerik yang dipakai: 379


Keputusan penggunaan fitur numerik saja

Fitur pada dataset terdiri dari gabungan fitur numerik dan beberapa fitur bertipe object. Untuk menjaga penggunaan memori dan menyederhanakan pipeline, pada tugas ini saya hanya menggunakan fitur numerik.

Untuk fitur numerik, nilai hilang diisi dengan median lalu diskalakan menggunakan StandardScaler. Fitur non-numerik di-drop melalui parameter remainder="drop" pada ColumnTransformer. Pendekatan ini sudah cukup untuk menunjukkan alur preprocessing end-to-end dan dapat dikembangkan lagi dengan encoding fitur kategorik sebagai pekerjaan lanjutan.

# Fungsi evaluasi model

Fungsi ini menghitung Accuracy, Precision, Recall, F1, ROC AUC dan menampilkan confusion matrix.

In [6]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)

def evaluate_model(name, model, X_valid, y_valid):
    y_proba = model.predict_proba(X_valid)[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)

    acc = accuracy_score(y_valid, y_pred)
    prec = precision_score(y_valid, y_pred, zero_division=0)
    rec = recall_score(y_valid, y_pred, zero_division=0)
    f1 = f1_score(y_valid, y_pred, zero_division=0)
    roc = roc_auc_score(y_valid, y_proba)
    cm = confusion_matrix(y_valid, y_pred)

    print(f"\n=== {name} ===")
    print("Accuracy :", acc)
    print("Precision:", prec)
    print("Recall   :", rec)
    print("F1-score :", f1)
    print("ROC AUC  :", roc)
    print("Confusion matrix:")
    print(cm)

    return {
        "model": name,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": roc
    }


# Model baseline Logistic Regression

Pemilihan model dan penanganan class imbalance

Label isFraud sangat tidak seimbang. Jumlah transaksi normal jauh lebih banyak dibanding transaksi fraud. Jika model dilatih tanpa penyesuaian, model cenderung bias ke kelas mayoritas.

Untuk mengatasi hal ini, saya menggunakan parameter class_weight="balanced" pada LogisticRegression dan RandomForest. Parameter ini otomatis memberi bobot lebih besar pada kelas minoritas (fraud) sehingga model lebih memperhatikan transaksi fraud saat proses training.

Sebagai baseline, saya menggunakan Logistic Regression. Lalu saya menambahkan RandomForest sebagai model tree-based yang mampu menangkap hubungan non-linear antar fitur.

Model linear sederhana.
Pakai class_weight="balanced" untuk menangani class imbalance.

In [7]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    class_weight="balanced",
    solver="saga"
)

log_reg_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", log_reg)
])

print("Training Logistic Regression...")
log_reg_pipeline.fit(X_train, y_train)

results = []
res_lr = evaluate_model("Logistic Regression", log_reg_pipeline, X_valid, y_valid)
results.append(res_lr)


Training Logistic Regression...


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



=== Logistic Regression ===
Accuracy : 0.8373666666666667
Precision: 0.10789019005567288
Recall   : 0.707808564231738
F1-score : 0.18723971347659504
ROC AUC  : 0.8594807129620894
Confusion matrix:
[[24559  4647]
 [  232   562]]


# Model baseline RandomForest

Model tree based, cocok untuk data tabular.
Juga gunakan class_weight="balanced".

In [8]:
from sklearn.ensemble import RandomForestClassifier

results = results  # lanjut dari sebelumnya

# baseline
rf_base = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    n_jobs=-1,
    class_weight="balanced",
    random_state=42
)

rf_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", rf_base)
])

print("\nTraining RandomForest baseline...")
rf_pipeline.fit(X_train, y_train)
res_rf_base = evaluate_model("RandomForest baseline", rf_pipeline, X_valid, y_valid)
results.append(res_rf_base)


Training RandomForest baseline...

=== RandomForest baseline ===
Accuracy : 0.9826
Precision: 0.9415584415584416
Recall   : 0.36523929471032746
F1-score : 0.5263157894736842
ROC AUC  : 0.9043994100104684
Confusion matrix:
[[29188    18]
 [  504   290]]


# Tuning sederhana hyperparameter RandomForest

Coba beberapa kombinasi n_estimators dan max_depth.
Pilih kombinasi dengan ROC AUC terbaik.

In [9]:
param_grid = [
    {"n_estimators": 100, "max_depth": 10},
    {"n_estimators": 200, "max_depth": 10},
    {"n_estimators": 200, "max_depth": 20},
]

best_model = None
best_score = -1
best_cfg = None

for cfg in param_grid:
    print(f"\nTraining RF n={cfg['n_estimators']} depth={cfg['max_depth']}")

    rf = RandomForestClassifier(
        n_estimators=cfg["n_estimators"],
        max_depth=cfg["max_depth"],
        n_jobs=-1,
        class_weight="balanced",
        random_state=42
    )

    rf_pipe = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", rf)
    ])

    rf_pipe.fit(X_train, y_train)
    res = evaluate_model(
        f"RandomForest n={cfg['n_estimators']} depth={cfg['max_depth']}",
        rf_pipe,
        X_valid,
        y_valid
    )
    results.append(res)

    if res["roc_auc"] > best_score:
        best_score = res["roc_auc"]
        best_model = rf_pipe
        best_cfg = cfg

import pandas as pd
results_df = pd.DataFrame(results)
print("\nRingkasan metrik:")
display(results_df)

print("\nKonfigurasi terbaik:", best_cfg)
print("ROC AUC terbaik:", best_score)


Training RF n=100 depth=10

=== RandomForest n=100 depth=10 ===
Accuracy : 0.9149333333333334
Precision: 0.1913623595505618
Recall   : 0.6863979848866498
F1-score : 0.299286106534871
ROC AUC  : 0.8916032875822933
Confusion matrix:
[[26903  2303]
 [  249   545]]

Training RF n=200 depth=10

=== RandomForest n=200 depth=10 ===
Accuracy : 0.9157666666666666
Precision: 0.1930570315267446
Recall   : 0.6863979848866498
F1-score : 0.30135471385125795
ROC AUC  : 0.8917726956832823
Confusion matrix:
[[26928  2278]
 [  249   545]]

Training RF n=200 depth=20

=== RandomForest n=200 depth=20 ===
Accuracy : 0.9742333333333333
Precision: 0.5133079847908745
Recall   : 0.5100755667506297
F1-score : 0.5116866708780796
ROC AUC  : 0.9052012362112543
Confusion matrix:
[[28822   384]
 [  389   405]]

Ringkasan metrik:


,model,accuracy,precision,recall,f1,roc_auc
0,Logistic Regression,0.837367,0.107890,0.707809,0.187240,0.859481
1,RandomForest baseline,0.982600,0.941558,0.365239,0.526316,0.904399
2,RandomForest n=100 depth=10,0.914933,0.191362,0.686398,0.299286,0.891603
3,RandomForest n=200 depth=10,0.915767,0.193057,0.686398,0.301355,0.891773
4,RandomForest n=200 depth=20,0.974233,0.513308,0.510076,0.511687,0.905201



Konfigurasi terbaik: {'n_estimators': 200, 'max_depth': 20}
ROC AUC terbaik: 0.9052012362112543


Tuning sederhana hyperparameter

Saya melakukan tuning sederhana pada RandomForest dengan mencoba beberapa kombinasi hyperparameter n_estimators dan max_depth. Untuk setiap kombinasi, model dilatih di data training dan dievaluasi pada data validasi menggunakan beberapa metrik. Sebagai kriteria utama, saya menggunakan nilai ROC AUC karena metrik ini mengukur kemampuan model membedakan kelas fraud dan non-fraud di berbagai threshold.

Dari hasil evaluasi, saya memilih konfigurasi RandomForest dengan nilai ROC AUC terbaik sebagai kandidat model akhir. Selain ROC AUC, saya juga melihat Recall karena dalam kasus fraud detection penting untuk mengurangi transaksi fraud yang lolos.

# Train final model di seluruh data train dan prediksi data test

Final model adalah RandomForest dengan konfigurasi terbaik.
Kita latih ulang di seluruh data train, lalu prediksi probabilitas fraud di test.

In [10]:
# final model dari tuning
final_model = best_model

print("\nTraining final model di seluruh data sample...")
final_model.fit(X, y)

# Prediksi probabilitas fraud di test sample
test_proba = final_model.predict_proba(X_test)[:, 1]

submission = pd.DataFrame({
    "TransactionID": test_ids,
    "isFraud": test_proba
})

print("Contoh isi submission:")
display(submission.head())


Training final model di seluruh data sample...
Contoh isi submission:


,TransactionID,isFraud
0,3663549,0.063600
1,3663550,0.095635
2,3663551,0.145115
3,3663552,0.021589
4,3663553,0.008793


Prediksi probabilitas fraud dan file submission

Model akhir dilatih ulang menggunakan seluruh data training yang tersedia pada subset. Untuk data test_transaction, model menghasilkan nilai probabilitas untuk kelas fraud (isFraud=1) melalui fungsi predict_proba.

Probabilitas ini yang saya simpan sebagai kolom isFraud di file submission bersama dengan TransactionID. Format submission adalah dua kolom: TransactionID dan isFraud, sesuai dengan deskripsi tugas. Nilai isFraud di sini adalah probabilitas, bukan label kelas 0 atau 1.

# Simpan submission ke CSV

In [11]:
submission_path = "/content/submission_fraud_rf_final.csv"
submission.to_csv(submission_path, index=False)
print("File submission disimpan sebagai:", submission_path)


File submission disimpan sebagai: /content/submission_fraud_rf_final.csv
